In [ ]:
!git clone https://github.com/AleSilva04/wv72_tf_201822717_201620127_201816502_20181d073_201815776.git

In [43]:
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import LeerCsv as lee
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import Generar_Grafo as ga
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import Funciones_Agrupar as ag

In [44]:
G = ga.generateGridGraph(1000)

In [45]:
almacen = lee.leer_archivo("wv72_tf_201822717_201620127_201816502_20181d073_201815776/dataset/warehouses.csv")
entrega = lee.leer_archivo("wv72_tf_201822717_201620127_201816502_20181d073_201815776/dataset/delivery-points.csv")

In [46]:
almacenes = []
for x,y in almacen:
  almacenes.append(y*1000+x)

ptsEntrega = []

for x,y in entrega:
  ptsEntrega.append(y*1000 +x)

In [47]:
#Le asignamos un almacen a cada uno de los 5000 puntos de entrega

def dist(a,b):
  x1,y1 = a
  x2,y2 = b
  return abs(x1-x2) + abs(y1-y2)

group =[-1]*5000
for i in range(5000):
  m=0
  d=dist(entrega[i],almacen[0])
  for j in range(1,100):
    d2 = dist(entrega[i],almacen[j])
    if d2 < d:
      d=d2
      m = j
  group[i]=m

In [48]:
#Agrupamos todos los nodos en su correspondiente grupo  
groupNodes = [[] for _ in range(100)]
for i in range(5000):
  g = group[i]
  groupNodes[g].append(ptsEntrega[i])

In [49]:
#Dividimos el Grafo General en subgrafos o regiones donde en cada una estan un almacen y sus respectivos puntos de entrega asignados
regions = [[] for _ in range(100)]
for i in range(100):
  g = groupNodes[i].copy()
  g.append(almacenes[i])
  limits = ag.limites(g)
  for y in range(limits[0],limits[1]+1):
    for x in range(limits[2],limits[3]+1):
      regions[i].append(y*1000+x)

In [50]:
#Modificamos el dijkstra, pero lo modificamos para utilizar diccionario
import heapq as hq
import math
def dijkstra(M,Pts, G, s):
  visited = {node:False for node in M}
  visitedPts = []
  path = {node:None for node in M}
  cost = {node:math.inf for node in M}
  cost[s] = 0
  queue = [(0, s)]
  while queue:
    if len(visitedPts) == len(Pts):
      break
    g_u, u = hq.heappop(queue)
    if visited[u] is False:
      visited[u] = True
      if u in Pts:
        visitedPts.append(1)
      for v in G[u]:
        if v not in M:
          continue
        f = g_u + 1
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (f, v))

  return path, cost

In [ ]:
#Aplicamos dijkstra para cada subgrafo, partiendo desde el almacen a todos los puntos de entrega

paths_costs=[([],[]) for _ in range(100)]
for i in range(100):
  g=groupNodes[i].copy()
  g.append(almacenes[i])
  path, cost = dijkstra(regions[i],g,G,almacenes[i])
  paths_costs[i]=(path,cost)
  print("termina ",i)

In [53]:
#Es la funcion para recorrer las rutas hacia un determinado punto de entrega
def recorrido(f, parent): 
  camino = [f]
  while f != None:
    a = parent[f]
    f = a
    camino.append(f)
  
  camino.pop(-1) 
  return camino
  
#Es la funcion donde Extraemos todos los caminos generados por el dikstra desde un determinado almacen
def recorridos(groupNodes,paths_costs,s): 
  parent, cost = paths_costs[s]
  destinos = groupNodes[s]
  caminos = []
  for destino in destinos:
    camino = recorrido(destino,parent)
    caminos.append(camino)
  return caminos 

In [54]:
#Aqui se obtiene todos las rutas necesarias para un subgrafo
#pues de su almacen llega a todos los puntos de entrega
def VRP(caminos, destinos,individual = False):

  rutas = []
  costos = []
  #Mientras aún haya destinos o grupos de entrega por llegar, que siga buscando rutas
  while destinos:
    encontrados = []
    cont = 0
    max = -1

    for camino in caminos:
      n = len(camino)
      cont = 0
      encuentra = []
      for punto in camino:
        if punto in destinos:
          cont += 1
          encuentra.append(punto)
      if max < cont:
        max = cont
        destinos_llegados = encuentra
        caminito = camino
    
    rutas.append(caminito)
    costos.append(len(caminito)-1)

    for dest in destinos_llegados:
      if dest in destinos:
        destinos.remove(dest)
  
  return rutas, costos

In [55]:
def imprimir(rutas,costos):
  n = len(rutas)
  total_costo = sum(costos)
  print("Número de rutas: ",n)
  for i in range(n):
    ruta_vuelta = list(reversed(rutas[i]))
    print("Ruta #",i,":",ruta_vuelta)
    print("Costo #",i,":",costos[i])

In [58]:
def VRP_total(groupNodes, paths_costs):
  for s in range(52,70): #Aqui podemos poner hasta el número 100, pues son 100 almacenes
    caminos = recorridos(groupNodes.copy(),paths_costs,s)
    rutas, costos = VRP(caminos,groupNodes[s].copy())
    print("Almacen #", s)
    imprimir(rutas,costos)
    print()

In [59]:
VRP_total(groupNodes, paths_costs)

Almacen # 52
Número de rutas:  3
Ruta # 0 : [662991, 662990, 662989, 662988, 662987, 662986, 662985, 662984, 662983, 662982, 662981, 663981, 664981, 665981]
Costo # 0 : 13
Ruta # 1 : [662991, 662992, 662993, 662994, 662995, 663995, 664995, 665995, 666995, 667995, 668995, 669995, 670995, 671995, 672995, 673995, 674995, 675995, 676995, 677995, 678995, 679995, 680995, 681995, 682995, 683995, 684995, 685995, 686995, 687995, 688995]
Costo # 1 : 30
Ruta # 2 : [662991, 662990, 662989, 662988, 662987, 662986, 662985, 662984, 662983, 662982, 663982, 664982, 665982, 666982, 667982, 668982, 669982, 670982, 671982, 672982, 673982]
Costo # 2 : 20

Almacen # 53
Número de rutas:  36
Ruta # 0 : [710214, 710213, 711213, 712213, 713213, 714213, 715213, 716213, 717213, 718213, 719213, 720213, 721213, 722213, 723213, 724213, 725213, 726213, 727213, 728213, 729213, 730213, 731213, 732213, 733213, 734213, 735213, 736213, 737213, 738213, 739213, 740213, 741213, 742213, 743213, 744213, 745213, 746213, 747213,